In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.tv_indicators import *
from quantfreedom.plotting.plotting_base import *
from quantfreedom.exchanges.exchange import Exchange
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash
from dash_bootstrap_templates import load_figure_template
from jupyter_dash import JupyterDash
from IPython import get_ipython
import dash_bootstrap_components as dbc

load_figure_template("darkly")
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
try:
    shell = str(get_ipython())
    if "ZMQInteractiveShell" in shell:
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    elif shell == "TerminalInteractiveShell":
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    else:
        app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
except NameError:
    app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])

bg_color = "#0b0b18"

mufex_main = Mufex(use_test_net=False)

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(exchange="binance_usdm", symbol="BTCUSDT", timeframe="30m", candles_to_dl=400)
open = candles[:, 1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]
close_shift = np.roll(candles[:, 4], 1)

In [10]:
rsi = rsi_tv_calc(source=close, length=14)

In [11]:
rsi

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, 67.81, 64.85, 69.56, 64.08, 61.94, 62.01, 63.82, 61.78, 59.85,
       55.17, 64.02, 62.39, 63.81, 67.39, 60.52, 61.25, 65.29, 60.56,
       56.79, 58.32, 54.11, 54.66, 52.41, 53.10, 55.90, 53.09, 50.72,
       54.15, 53.67, 48.56, 51.54, 52.95, 49.71, 53.90, 56.03, 53.50,
       57.22, 53.41, 50.84, 52.32, 54.20, 63.76, 56.60, 66.63, 75.69,
       67.29, 66.81, 59.61, 59.81, 58.57, 64.04, 56.21, 60.57, 62.18,
       69.94, 64.96, 60.82, 61.47, 61.45, 57.34, 54.26, 57.75, 59.77,
       57.40, 50.33, 52.98, 54.70, 52.17, 56.19, 56.48, 53.01, 56.22,
       51.83, 50.30, 36.81, 41.98, 41.57, 42.95, 41.39, 44.69, 47.21,
       56.97, 54.19, 51.96, 51.86, 55.70, 48.18, 54.51, 49.74, 42.97,
       33.21, 40.22, 50.21, 55.30, 51.37, 52.06, 53.34, 54.38, 45.43,
       44.29, 46.86, 46.58, 50.63, 46.54, 45.95, 41.21, 43.55, 44.10,
       46.31, 45.29, 49.59, 48.32, 52.75, 57.65, 61.11, 63.26, 61.97,
       62.48,

In [5]:
np.where(~np.isnan(rsi_tv_calc(source=close, length=14)))

(array([ 14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
        183, 184, 185, 186, 187, 188, 189, 190, 191

In [12]:
stdev_tv_calc(source=rsi, length=20)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, 3.43, 3.28, 3.26, 2.77, 2.76, 3.09, 3.21, 3.65, 3.96, 4.45,
       4.63, 4.55, 4.69, 4.82, 4.00, 3.90, 4.04, 2.98, 2.39, 2.40, 1.93,
       2.09, 2.04, 2.33, 2.33, 2.27, 2.27, 2.22, 3.56, 3.63, 4.59, 6.87,
       7.26, 7.21, 7.06, 7.00, 6.79, 6.80, 6.62, 6.01, 5.44, 5.27, 5.29,
       5.00, 4.97, 3.63, 3.58, 3.75, 3.83, 3.83, 3.88, 4.56, 4.80, 4.93,
       5.13, 3.99, 3.37, 3.28, 2.92, 2.61, 2.79, 5.35, 5.92, 6.10, 6.13,
       6.50, 6.50, 6.28, 6.62, 6.47, 6.16, 6.10, 6.05, 5.94, 6.18, 5.37,
       5.29, 6.43, 6.62, 6.37, 6.48, 6.47, 6.19, 6.15, 6.26, 6.30, 6.12,
       6.13, 5.88, 5.91, 5.78, 4.17, 4.03, 4.23, 3.90, 3.76, 3.54, 3.15,
       2.35, 2.91, 3.99, 5.30, 6.47, 7.20, 7.72, 8.32, 8.22, 7.86, 7.47,
       7.16, 6.44, 6.05, 5.43, 5.55, 6.29, 7.03, 7.44, 7.57, 8.00, 7.32,
       6.16, 4.99, 5.11, 5.12, 4.82, 3.93, 3.63, 4.07,

In [13]:
np.where(~np.isnan(stdev_tv_calc(source=rsi, length=20)))

(array([ 27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
        183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
        196, 197, 198, 199, 200, 201, 202, 203, 204

In [15]:
np.where(~np.isnan(stdev_tv_calc(source=close, length=20)))

(array([], dtype=int64),)

In [16]:
stdev_tv_calc(source=close, length=50)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na